#Notebook de importação do historico

In [0]:
%run "./library/config"

In [0]:
from pyspark.sql.functions import *

In [0]:
spark.conf.set(
    "fs.azure.account.key.sadatalake002poc2022.dfs.core.windows.net",
    acesskey
)

In [0]:
dbutils.fs.ls("abfss://poc@sadatalake002poc2022.dfs.core.windows.net/raw")

In [0]:
df_departments = spark.read.csv("abfss://poc@sadatalake002poc2022.dfs.core.windows.net/raw/departments.csv")
display(df_departments)

In [0]:
df_departments = (df_departments
                          .withColumnRenamed('_c0','id')
                          .withColumnRenamed('_c1','departments')                  
                 )
display(df_departments)

In [0]:
df_hemployees = spark.read.csv("abfss://poc@sadatalake002poc2022.dfs.core.windows.net/raw/hired_employees.csv")
display(df_hemployees)

In [0]:
df_hemployees = (df_hemployees
                    .withColumnRenamed('_c0','id')
                    .withColumnRenamed('_c1','name')
                    .withColumnRenamed('_c2','datetime')
                    .withColumnRenamed('_c3','department_id')
                    .withColumnRenamed('_c4','job_id')
                )
display(df_hemployees)

In [0]:
df_jobs = spark.read.csv("abfss://poc@sadatalake002poc2022.dfs.core.windows.net/raw/jobs.csv")
display(df_jobs)

In [0]:
df_jobs = (df_jobs
              .withColumnRenamed('_c0','id')
              .withColumnRenamed('_c1','job')
          )
display(df_jobs)

In [0]:
jdbcUser = "sqladmin"
jdbcPassword  = pass_database
jdbcHostname = host_name
jdbcDatabase = "poc2022"
jdbcPort = 1433
jdbcDriver = "com.microsoft.sqlserver.jdbc.SQLServerDriver"

jdbcUrl = "jdbc:sqlserver://{0}:{1};database={2}".format(jdbcHostname, jdbcPort, jdbcDatabase)
connectionProperties = {
  "user" : jdbcUser,
  "password" : jdbcPassword,
  "driver" : jdbcDriver
}

In [0]:
table = "tb_jobs"

df_jobs.write.mode("overwrite") \
      .format("jdbc") \
      .option("url", jdbcUrl) \
      .option("dbtable", table) \
      .option("user", jdbcUser) \
      .option("password", jdbcPassword) \
      .option("driver", jdbcDriver) \
      .save()

In [0]:
spark.read.jdbc(url=jdbcUrl,table="tb_jobs",properties=connectionProperties).show()